<a href="https://colab.research.google.com/github/francesco-mollica/word2vec-entity_embeddings/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"

CHOI_path = base_dir + "corpus_data_CHOI.txt"
FIGER_path = base_dir + "corpus_data_FIGER.txt"
ontonotes_path = base_dir + "corpus_data_ontonotes.txt"
balanced_ontonotes_path = base_dir + "corpus_data_balanced_ontonotes.txt"
bbn_path = base_dir + "corpus_data_bbn.txt"

Mounted at /content/drive/


In [4]:
import pickle
with open(balanced_ontonotes_path, "rb") as fp:
  b = pickle.load(fp)

In [5]:
print(b)

['/PERSON', 'that', 'was', ',', 'the', 'only', 'way', 'you', 'could', 'overcome', 'them', 'and', 'get', 'the', 'upper', 'hand', '.']


In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.init_sims(replace=True)